In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras import applications
from datetime import datetime as dt
from keras import regularizers as reg
from keras.optimizers import RMSprop
from keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore")
from keras.callbacks import ModelCheckpoint
from sklearn.externals import joblib
import os
import gc

### Utility function to plot Confusion Matrix

In [2]:
#Utility function to plot the confusion matrices
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
def plot_confusion_matrix(cm_df, classes, normalize, title):
    if normalize:
        cm = cm_df.values
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        plt.figure(figsize = (7,7))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Greens)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=90)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")
        plt.tight_layout()
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
    
    else:
        import seaborn as sn
        plt.figure(figsize = (6,5))
        ax = sn.heatmap(cm_df, annot=True, fmt='d', cmap=plt.cm.Blues)   #fmt='d' for decimal integer.
        ax.set_xlabel("Predicted Labels")
        ax.set_ylabel("True Labels")
        ax.set_title(title)

#Utility function to design the confusion matrix DF
def get_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    cm_df =  pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])
    return cm_df

### Declaring general parameters from the model

In [3]:
#Dimensions of our flicker images is 256 X 256
img_width, img_height = 256, 256

#Declaration of parameters needed for training and validation
test_data_dir = 'data/test'
epochs = 50
batch_size = 20

### Declaring generator function for the test batches

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,
                                                shuffle=False)

nb_test_samples = len(test_generator.filenames) #2500. 500 testing samples for each class  
nb_validation_samples = len(test_generator.filenames) #2500. 500 testing samples for each class
num_classes = len(test_generator.class_indices)

Found 2500 images belonging to 5 classes.


### Extract bottleneck features from test batches

In [5]:
os.mkdir("cnn_codes") if not os.path.isdir("cnn_codes") else None

#Get the bottleneck features by  Weights.T * Xi
def get_test_bottlebeck_features(model_name):
    
    start=dt.now()
    
    #Load the pre trained model from Keras, we will initialize only the convolution layers and ignore the top layers.
    if model_name == "VGG16":
        model = applications.VGG16(include_top=False, weights='imagenet')
    elif model_name == "InceptionV3":
        model = applications.InceptionV3(include_top=False, weights='imagenet')
    elif model_name == "ResNet50": 
        model = applications.ResNet50(include_top=False, weights='imagenet')
    elif model_name == "InceptionResNetV2": 
        model = applications.InceptionResNetV2(include_top=False, weights='imagenet')
    elif model_name == "DenseNet201": 
        model = applications.DenseNet201(include_top=False, weights='imagenet')

    bottleneck_features_test = model.predict_generator(test_generator, nb_test_samples // batch_size)
    np.save('cnn_codes/{}_bottleneck_features_test.npy'.format(model_name),bottleneck_features_test) #bottleneck_features_test is a numpy array
    
    gc.collect()
    
    print("Got the test bottleneck features for {} model, in time: ".format(model_name),dt.now()-start)

In [8]:
#Get the test bottleneck features for all the models we have tried
get_test_bottlebeck_features("InceptionV3")
get_test_bottlebeck_features("VGG16")
get_test_bottlebeck_features("ResNet50")
get_test_bottlebeck_features("InceptionResNetV2")
get_test_bottlebeck_features("DenseNet201")

Got the test bottleneck features for InceptionV3 model, in time:  0:06:18.853306
Got the test bottleneck features for VGG16 model, in time:  0:23:09.425068
Got the test bottleneck features for ResNet50 model, in time:  0:09:26.531706
Got the test bottleneck features for InceptionResNetV2 model, in time:  0:13:40.346079
Got the test bottleneck features for DenseNet201 model, in time:  0:15:50.900298


### Load all the models

In [10]:
#Load the 3 CNN models
#Load the second CNN model
#Load the third CNN model

In [ ]:
#Load the pretrained models
inception_v3 = load_model('models/InceptionV3_bottleneck_features_test.h5')
vgg_16 = load_model('models/VGG16_bottleneck_features_test.h5')
res_net50 = load_model('models/ResNet50_bottleneck_features_test.h5')
inc_res_net_v2 = load_model('models/InceptionResNetV2_bottleneck_features_test.h5')
dense_net201 = load_model('models/DenseNet201_bottleneck_features_test.h5')



In [21]:
dense_net201 = load_model('models/densenet201_bottleneck_feats_multi_model.h5')

In [22]:
type(dense_net201)

keras.engine.sequential.Sequential

In [23]:
dense_net201.history

AttributeError: 'Sequential' object has no attribute 'history'

In [24]:
dense_net201.predict()

TypeError: predict() missing 1 required positional argument: 'x'